Задание 1. Обработка массива на CPU (OpenMP)

In [9]:
%%writefile cpu_process.cpp

#include <iostream>     // ввод-вывод
#include <vector>       // контейнер std::vector
#include <chrono>       // измерение времени
#include <omp.h>        // OpenMP для параллельных вычислений на CPU

// Функция обработки массива на CPU
// Каждый элемент массива умножается на 2
void cpu_process(std::vector<float>& data) {

    // Директива OpenMP
    // parallel for автоматически распараллеливает цикл
    // Итерации цикла распределяются между потоками CPU
    #pragma omp parallel for
    for (size_t i = 0; i < data.size(); i++) {
        data[i] *= 2.0f;
    }
}

int main() {
    // Размер массива
    const size_t N = 1000000;

    // Инициализация массива значениями 1.0
    std::vector<float> data(N, 1.0f);

    // Засекаем время начала вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // Обработка массива на CPU с использованием OpenMP
    cpu_process(data);

    // Засекаем время окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();

    // Вычисляем длительность выполнения в миллисекундах
    std::chrono::duration<double, std::milli> time = end - start;

    // Вывод времени выполнения
    std::cout << "CPU time: " << time.count() << " ms\n";

    return 0;
}


Overwriting cpu_process.cpp


In [10]:
!g++ -o cpu_process cpu_process.cpp -lstdc++ -fopenmp
!./cpu_process

CPU time: 3.44847 ms


Задание 2. Обработка массива на GPU (CUDA)

In [1]:
%%writefile gpu_process.cu

#include <cuda_runtime.h>   // основные функции CUDA
#include <iostream>         // ввод-вывод
#include <vector>           // контейнер std::vector
#include <chrono>           // измерение времени

// CUDA-ядро (kernel)
// Выполняется на GPU
// Каждый поток обрабатывает один элемент массива
__global__ void gpu_kernel(float* data, int N) {

    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < N) {
        data[idx] *= 2.0f;
    }
}

int main() {

    // Размер массива
    const int N = 1'000'000;

    // Размер памяти в байтах
    size_t size = N * sizeof(float);

    // Массив в памяти хоста (CPU)
    std::vector<float> h_data(N, 1.0f);

    // Указатель на массив в памяти устройства (GPU)
    float* d_data;

    // Выделение памяти на GPU
    cudaMalloc(&d_data, size);

    // Копирование данных с CPU на GPU
    cudaMemcpy(d_data, h_data.data(), size, cudaMemcpyHostToDevice);

    // Засекаем время начала вычислений на GPU
    auto start = std::chrono::high_resolution_clock::now();

    // Количество потоков в одном блоке
    int threads = 256;

    // Количество блоков в сетке
    int blocks = (N + threads - 1) / threads;

    // Запуск CUDA-ядра
    gpu_kernel<<<blocks, threads>>>(d_data, N);

    // Ожидание завершения всех потоков GPU
    cudaDeviceSynchronize();

    // Засекаем время окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();

    // Копирование результатов с GPU обратно на CPU
    cudaMemcpy(h_data.data(), d_data, size, cudaMemcpyDeviceToHost);

    // Освобождение памяти GPU
    cudaFree(d_data);

    // Вычисление времени выполнения в миллисекундах
    std::chrono::duration<double, std::milli> time = end - start;

    // Вывод времени выполнения GPU-вычислений
    std::cout << "GPU time: " << time.count() << " ms\n";

    return 0;
}

Writing gpu_process.cu


In [2]:
!nvcc -o gpu_process gpu_process.cu -lstdc++ -lcudart
!./gpu_process

GPU time: 50.5873 ms


Задание 3. Гибридная обработка массива

In [5]:
%%writefile hybrid_process.cu

#include <iostream>        // ввод-вывод
#include <vector>          // std::vector
#include <chrono>          // измерение времени
#include <omp.h>           // OpenMP для параллельных вычислений на CPU
#include <cuda_runtime.h>  // CUDA API

// CUDA-ядро
// Обрабатывает вторую половину массива, начиная с offset
__global__ void gpu_kernel(float* data, int offset, int N) {

    // Глобальный индекс потока внутри GPU
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка границ
    if (idx < N) {
        data[offset + idx] *= 2.0f;
    }
}

int main() {

    // Общий размер массива
    const int N = 1'000'000;

    // Половина массива для гибридной обработки
    const int half = N / 2;

    // Размер памяти в байтах
    size_t size = N * sizeof(float);

    // Массив в памяти CPU
    std::vector<float> data(N, 1.0f);

    // Указатель на массив в памяти GPU
    float* d_data;

    // Выделение памяти на GPU
    cudaMalloc(&d_data, size);

    // Копирование всего массива с CPU на GPU
    cudaMemcpy(d_data, data.data(), size, cudaMemcpyHostToDevice);

    // Начало измерения общего времени гибридных вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // Параллельные секции OpenMP
    // CPU и GPU работают одновременно
    #pragma omp parallel sections
    {
        // Первая секция: обработка первой половины массива на CPU
        #pragma omp section
        {
            for (int i = 0; i < half; i++) {
                data[i] *= 2.0f;
            }
        }

        // Вторая секция: обработка второй половины массива на GPU
        #pragma omp section
        {
            int threads = 256;                          // потоки в блоке
            int blocks = (half + threads - 1) / threads; // количество блоков

            // Запуск CUDA-ядра для второй половины массива
            gpu_kernel<<<blocks, threads>>>(d_data, half, half);

            // Ожидание завершения вычислений на GPU
            cudaDeviceSynchronize();
        }
    }

    // Копирование обработанной второй половины массива с GPU на CPU
    cudaMemcpy(data.data() + half,
               d_data + half,
               half * sizeof(float),
               cudaMemcpyDeviceToHost);

    // Освобождение памяти GPU
    cudaFree(d_data);

    // Окончание измерения времени
    auto end = std::chrono::high_resolution_clock::now();

    // Общее время гибридной обработки
    std::chrono::duration<double, std::milli> time = end - start;

    // Вывод времени выполнения
    std::cout << "Hybrid time: " << time.count() << " ms\n";

    return 0;
}


Overwriting hybrid_process.cu


In [6]:
!nvcc -o hybrid_process hybrid_process.cu -lstdc++ -lcudart
!./hybrid_process

Hybrid time: 9.6639 ms




## Задание 4. Анализ производительности

В ходе эксперимента были получены следующие результаты времени выполнения обработки массива размером 1 000 000 элементов:

* CPU (OpenMP): **3.45 мс**
* GPU (CUDA): **50.59 мс**
* Гибридный режим (CPU + GPU): **9.66 мс**

### Сравнение результатов

Наименьшее время выполнения показала реализация на CPU с использованием OpenMP. Это объясняется тем, что операция умножения элементов массива является достаточно простой, а накладные расходы на передачу данных между CPU и GPU в данном случае превышают выигрыш от параллельных вычислений на GPU.

GPU-реализация оказалась самой медленной. Основная причина — необходимость копирования данных между памятью CPU и GPU по шине PCI Express, а также синхронизация выполнения (`cudaDeviceSynchronize`). Для такой простой операции стоимость этих действий значительно выше, чем само вычисление.

Гибридная реализация показала промежуточный результат. За счет параллельной работы CPU и GPU общее время выполнения уменьшилось по сравнению с чистой GPU-реализацией, однако гибридный вариант все равно уступает CPU-реализации из-за накладных расходов, связанных с использованием GPU.

### Анализ эффективности гибридного подхода

Гибридный подход дает наибольший выигрыш в следующих случаях:

* при больших объемах данных, когда время вычислений на GPU значительно превышает время передачи данных;
* при вычислительно сложных операциях, а не простых арифметических действиях;
* когда возможно перекрыть вычисления на CPU и GPU, минимизируя время простоя;
* если данные уже частично находятся в памяти GPU и не требуют повторного копирования.

В рассматриваемом эксперименте гибридный подход не показал максимальной эффективности, так как задача слишком проста и ограничена накладными расходами на обмен данными между CPU и GPU.

### Общий вывод

Для простых операций над массивами оптимальным решением является параллельная обработка на CPU с использованием OpenMP. Использование GPU и гибридного подхода оправдано только при более сложных вычислениях и больших объемах данных, где преимущества массового параллелизма GPU перекрывают затраты на передачу данных.